## Задача 3. Регуляризация

1. Сгенерируйте случайную малоранговую матрицу $X$ $500 \times 100$ ранга $10$
2. Сгенерируйте случайный вектор $w$ размера $100$ и целевой вектор 
$$
y = Xw + \varepsilon,
$$
где $\varepsilon$ - вектор с элементами из нормального распределения с нулевым средним и дисперсией $0.01$. 
3. Сравните разреженность и точность решения, которое дают методы регуляризации рассмотренные на лекции. Сделайте вывод об их эффективности. Рассмотрите, как автоматический подбор параметра регуляризации, так и эвристический подбор. Сравните эффективность методов. 
4. Проведите аналогичный эксперимент, но на данных, в которых целевой вектор сгенерирован так, что он хорошо аппроксимируется как одним признаком, так и линейной комбинацией двух или трёх других признаков. Выделяют ли методы регуляризации подобные зависимости в данных? Почему?
5. Добавьте в сгенерированную выборку несколько (5-10) выбросов, то есть таких строк и соответствующих им значений в $y$, что они не подчиняются ранее заданному закону.
6. Проверьте как изменится точность решения задачи для ранее рассмотренных методов
7. Сравните предыдущие методы регуляризации с [регуляризацией Huber'a](https://en.wikipedia.org/wiki/Huber_loss). Какой способ устойчивее к выбросам и почему?
8. Получите аналитическое решение задачи
$$
\min_w \|Xw - y\|^2_2 + \| \lambda \odot w \|^2_2,
$$
где $\lambda$ - вектор такой же размерности, что и $w$, $\odot$ означает поэлементное умножение двух векторов

Генерация случайной малоранговой матрицы $X$ $500\times100$ ранга 10

In [66]:
import numpy as np

def generateLowRankMatrix(rowsCount, columnCount, rank):
    matrix = np.zeros((rowsCount, columnCount))
    for i in range(rank):
        a = np.random.rand(rowsCount, 1)
        b = np.random.rand(1, columnCount)
        matrix = matrix + a.dot(b)
    return matrix

In [88]:
rowsCount = 500
columnCount = 100
rank = 10
matrix = generateLowRankMatrix(rowsCount, columnCount, rank)
# matrix_U = np.random.rand(rank, rowsCount)
# matrix_V = np.random.rand(rank, columnCount)
# matrix = matrix_U.T.dot(matrix_V)

Генерация случайного вектора $y = Xw + \varepsilon$

In [89]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import norm as normal_distribution
w = np.random.rand(columnCount, 1)

mean = 0
standartDeviation = 0.01
eps = np.random.normal(mean, standartDeviation, size = rowsCount)
epsr = eps.reshape(rowsCount, 1)

mw = matrix.dot(w)
y = matrix.dot(w) + epsr

Сравните разреженность и точность решения, которое дают методы регуляризации рассмотренные на лекции. Сделайте вывод об их эффективности. Рассмотрите, как автоматический подбор параметра регуляризации, так и эвристический подбор. Сравните эффективность методов.

In [90]:
from sklearn import linear_model as models
from numpy.linalg import norm
from IPython.display import HTML, display
import tabulate

def bold(s):
    return '<b>' + s + '</b>'

def modelStats(modelName, matrix, mw_c, y_c):
    
    mw = mw_c.ravel()
    y = y_c.ravel()
    
    #wtf no switch-case in python mdaa
    if modelName == "Lasso":
        model = models.Lasso(alpha = 1.0, max_iter = 10000)
    elif modelName == "Lasso CV":
        model = models.LassoCV(max_iter = 10000)
    elif modelName == "Ridge":
        model = models.Ridge(alpha = 1.0, max_iter = 10000)
    elif modelName == "Ridge CV":
        model = models.RidgeCV()
    elif modelName == "ElasticNet":
        model = models.ElasticNet(alpha=1.0, l1_ratio=0.1)
    elif modelName == "ElasticNet CV":
        model = models.ElasticNetCV()
    elif modelName == "Hubber":
        model = models.HuberRegressor()
        
    fit = model.fit(matrix, y)
    scoreNoise = fit.score(matrix, mw)
    score = fit.score(matrix, y)
    predict = norm(model.predict(matrix) - mw, 2)
    wNNZ = np.count_nonzero(model.coef_)
    return [bold(modelName), score, scoreNoise, predict, wNNZ]

table = [map(bold, ["Name", "Score (original)", "Score (with noise)", "||y_predict - y||", "wnnz"])]
table.append(modelStats("Lasso", matrix, mw, y))
table.append(modelStats("Lasso CV", matrix, mw, y))
table.append(modelStats("Ridge", matrix, mw, y))
table.append(modelStats("Ridge CV", matrix, mw, y))
table.append(modelStats("ElasticNet", matrix, mw, y))
table.append(modelStats("ElasticNet CV", matrix, mw, y))
table.append(modelStats("Hubber", matrix, mw, y))

def tableFullWidth(rawHTML):
    return '<style>table {width: 100%}</style>' + rawHTML

display(HTML(tableFullWidth(tabulate.tabulate(table, tablefmt='html'))))

Name,Score (original),Score (with noise),||y_predict - y||,wnnz
Lasso,0.995060358161,0.995057929024,38.2828946601,7
Lasso CV,0.999997562831,0.999997668911,0.83143769747,12
Ridge,0.999999825298,0.999999981125,0.0748160342711,100
Ridge CV,0.999999837208,0.999999997473,0.0273729746743,100
ElasticNet,0.997541451345,0.997539683719,27.0113163628,100
ElasticNet CV,0.999985489567,0.999985441888,2.0777955429,77
Hubber,0.999999837235,0.999999998469,0.0213108250144,100


Some analysis.... todo

Добавьте в сгенерированную выборку несколько (5-10) выбросов, то есть таких строк и соответствующих им значений в  yy , что они не подчиняются ранее заданному закону.

In [91]:
eps2 = eps.copy()
# print(eps2)
maxModifications = 25
for i in range(maxModifications):
    yi = np.random.randint(rowsCount)
    r = np.random.chisquare(2, size = 1)[0]
    eps2[yi] += 10.0 * r
epsr2 = eps2.reshape(rowsCount, 1)

y2 = matrix.dot(w) + epsr2

Проверьте как изменится точность решения задачи для ранее рассмотренных  методов и сравните предыдущие методы регуляризации с регуляризацией Huber'a. Какой способ устойчивее к выбросам и почему?

In [92]:
table2 = [map(bold, ["Name", "Score (original)", "Score (with noise)", "||y_predict - y||", "wnnz"])]
table2.append(modelStats("Lasso", matrix, mw, y2))
table2.append(modelStats("Lasso CV", matrix, mw, y2))
table2.append(modelStats("Ridge", matrix, mw, y2))
table2.append(modelStats("Ridge CV", matrix, mw, y2))
table2.append(modelStats("ElasticNet", matrix, mw, y2))
table2.append(modelStats("ElasticNet CV", matrix, mw, y2))
table2.append(modelStats("Hubber", matrix, mw, y2))
display(HTML(tableFullWidth(tabulate.tabulate(table2, tablefmt='html'))))

Name,Score (original),Score (with noise),||y_predict - y||,wnnz
Lasso,0.924251917628,0.9930617801,45.3601319688,8
Lasso CV,0.929013992076,0.996532680283,32.0661798333,14
Ridge,0.92901625928,0.996497215674,32.2297530666,100
Ridge CV,0.92901407125,0.996583170909,31.8318520259,100
ElasticNet,0.926300834361,0.996121941071,33.9123184138,100
ElasticNet CV,0.928815151914,0.99692350157,30.2049852614,97
Hubber,0.925835209356,0.999999996281,0.0332109254862,100


Функция потерь Хьюбера менее чувствительна к выбросам, чем все остальные методы